In [7]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, LlamaTokenizer
import torch
import json
import os

In [4]:
tokenizer = LlamaTokenizer.from_pretrained('/mnt/data/zoo/llama2/llama2-7b-hf/')
batch_size = 4
input_ids = tokenizer.encode('os:', return_tensors='pt')
# tokenizer(
#     # batch_size*[''],
#     ['4235 ', '5462 ', '7132 ', '6460 '], 
#     return_tensors="pt"
# ).input_ids  # Batch size 1

def sample(ckpt_dir, step):
    output_dir = os.path.join(ckpt_dir, f'checkpoint-{step}') #f'/mnt/data/sonia/ckpts/llama-2-7b-jul25/checkpoint-{step}/'
    print(output_dir)
    model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
    model = model.merge_and_unload() #returns type transformers.models.llama.modeling_llama.LlamaForCausalLM
    
    outputs = model.generate(input_ids, max_new_tokens=100).cpu()
    out=tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    model = model.cpu()
    return out

In [10]:
s = sample('/mnt/data/sonia/ckpts/debug', 20)
s

/mnt/data/sonia/ckpts/debug/checkpoint-20


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

['os:\n\n    # 1. 설정 파일 생성\n    # 2. 설정 파일 읽어오기\n    # 3. 설정 파일 쓰기\n    # 4. 설정 파일 삭제\n    # 5. 설정 �']

In [9]:
print(s[0])

os:
1 {'os': 'Windows Server 2012 R2 Datacenter 9600', 'ip_str': '10.10.10.10', 'port': 445, 'module': 'smb', 'cpe': None, 'cpe_count': 0, 'category': 'file_sharing', 'os_generic': 'windows server', 'single_cpe': None}


In [9]:
for lr in [-4,-6,-8]:
    for wd in ['1e-3', '1e-4', '1e-5']:
        for i in range(10,22,2):
            print(f'lr 1e{lr}, wd {wd}, epoch {i}')
            ckptdir = f'/mnt/data/sonia/ckpts/grid/lr1e{lr}_wd{wd}/checkpoint-{i}/adapter_model'
            for out in sample(i, ckptdir):
                print(out)
            print('')
        print('\n')

lr 1e-4, wd 1e-3, epoch 10


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

4235.0 - Population by Age and Sex, Regions of Australia, 2011
Population by Age and Sex, Regions of Australia, 2011
Population by Age and Sex, Regions of Australia, 2011 (cat. no. 4235.0) was released on 20 December 2012.
The ABS has released a new publication, Population by Age and Sex, Regions of
5462.5000 - 5462.5000
5462.5000 - 5462.5000 - 5462.5000
5462.5000 - 5462.5000 - 5462.5000
5462.5000 - 5462.500
7132.0 - Australian National Accounts: National Income, Expenditure and Product, Jun 2018 Quality Declaration
Australian National Accounts: National Income, Expenditure and Product, Jun 2018
The Australian National Accounts: National Income, Expenditure and Product, Jun 2018 (cat. no. 5206.0) publication presents estimates of the value of goods and services produced in Australia
6460000000000000000♠0.0000000000000000000♠0.000000000000000000♠0.000000000000000000♠0.000000000000000000

lr 1e-4, wd 1e-3, epoch 12


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

4235.0 - Population by Age and Sex, Regions of Australia, 2011
Population by Age and Sex, Regions of Australia, 2011
Population by Age and Sex, Regions of Australia, 2011 (cat. no. 4235.0) was released on 29 May 2013.
The ABS has released a new publication, Population by Age and Sex, Regions of
5462.5000 - 5462.5000
5462.5000 - 5462.5000 - 5462.5000
5462.5000 - 5462.5000 - 5462.5000
5462.5000 - 5462.500
7132.0 - Australian National Accounts: National Income, Expenditure and Product, Jun 2017 Quality Declaration
Australian National Accounts: National Income, Expenditure and Product, Jun 2017
The Australian National Accounts: National Income, Expenditure and Product (cat. no. 5206.0) is a comprehensive and detailed quarterly statistical publication which presents estimates of the economic performance of the Australian
6460000000000000000♠0.0000000000000000000♠0.000000000000000000♠0.000000000000000000♠0.000000000000000000

lr 1e-4, wd 1e-3, epoch 14


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:
tokenizer = LlamaTokenizer.from_pretrained('/mnt/data/zoo/llama2/llama2-7b-hf/')
batch_size = 4
input = tokenizer(
    # batch_size*[''],
    ['4235 ', '5462 ', '7132 ', '6460 '], 
    return_tensors="pt"
)  # Batch size 1
input

{'input_ids': tensor([[    1, 29871, 29946, 29906, 29941, 29945, 29871],
        [    1, 29871, 29945, 29946, 29953, 29906, 29871],
        [    1, 29871, 29955, 29896, 29941, 29906, 29871],
        [    1, 29871, 29953, 29946, 29953, 29900, 29871]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]])}

In [14]:
step=20
output_dir = f'/mnt/data/sonia/ckpts/debug/checkpoint-{step}/'
print(output_dir)
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
# model=model.merge_and_unload()

/mnt/data/sonia/ckpts/debug/checkpoint-20/


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
outputs = model.generate(**input, max_length=100)
out=tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)
out

/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


["4235  {'os': 'Windows Server 2008 R2 Datacenter 7601 Service Pack 1', 'ip_str': '10.1.1.1', 'port': 445, 'module': 'smb', 'cpe': None, 'cpe_count': 0, 'category': 'file_sharing', 'os_generic': 'windows server', 'single_cpe",
 "5462  {'os': 'Windows Server 2008 R2 Datacenter 7601 Service Pack 1', 'ip_str': '10.1.1.1', 'port': 445, 'module': 'smb', 'cpe': None, 'cpe_count': 0, 'category': 'file_sharing', 'os_generic': 'windows server', 'single_cpe",
 "7132  {'os': 'Windows Server 2008 R2 Datacenter 7601 Service Pack 1', 'ip_str': '10.1.1.1', 'port': 445, 'module': 'smb', 'cpe': None, 'cpe_count': 0, 'category': 'file_sharing', 'os_generic': 'windows server', 'single_cpe",
 "6460  {'os': 'Windows Server 2008 R2 Datacenter 7601 Service Pack 1', 'ip_str': '10.1.1.1', 'port': 445, 'module': 'smb', 'cpe': None, 'cpe_count': 0, 'category': 'file_sharing', 'os_generic': 'windows server', 'single_cpe"]